## Tests separating Norm Initiatives from aggregated PDFs

In [1]:
import re
import pandas as pd
import textract

In [151]:
def spa_char(string):
    string = re.sub(r'\\xc3\\xa1', r'á', string)
    string = re.sub(r'\\xc3\\xa9', r'é', string)
    string = re.sub(r'\\xc3\\xad', r'í', string)
    string = re.sub(r'\\xc3\\xb3', r'ó', string)
    string = re.sub(r'\\xc3\\xba', r'ú', string)
    string = re.sub(r'\\xc3\\xb1', r'ñ', string)
    string = re.sub(r'\\xef\\xac\\x81', r'fi', string)
    string = re.sub(r'\\xe2\\x80\\x9c', r'"', string)
    string = re.sub(r'\\xe2\\x80\\x9d', r'"', string)
    string = re.sub(r'\\xef\\xac\\x82', r'fl', string)
    string = re.sub(r'\\xc2\\xb0', r'fl', string)
    string = re.sub(r'\\xc3\\x93', r'Ó', string)
    string = re.sub(r'\\xc2\\xba', r'', string)
    string = re.sub(r'\\xc3\\x91', r'Ñ', string)
    string = re.sub(r'\\xe2\\x80\\x93', r'-', string)
    string = re.sub(r'\\xc3\\x81', r'Á', string)
    string = re.sub(r'\\xc2\\xa1', r'', string)
    return string

In [2]:
df = pd.read_csv('Data/ind_docs.csv')

In [3]:
mult = spa_char(str(textract.process('PDFs/Multiples/1.pdf', method='pdfminer')))

In [14]:
texts = re.findall(r'Construcción de la norma(.*?)Archivos Adjuntos', mult)

In [68]:
# Due to the formatting and extraction, sometimes the proposal topic is printed
# after the argumentation begins.
topic_regex = r'[\\]{1,2}n[\\]{1,2}n.+?[\\]{1,2}n\d\s-\s.+?[\\]{1,2}n[\\]n'
topic_regex2 = r'\d\s-\s'
nr_w_topic = 0
nr_no_topic = 0
nr_m_topic = 0
w_topic = []
for text in texts:
    topic = re.findall(topic_regex2, text)
    if len(topic) == 0:
        nr_no_topic += 1
    elif len(topic) == 1:
        nr_w_topic += 1
        w_topic.append(texts.index(text))
    else:
        nr_m_topic += 1
(nr_w_topic, nr_no_topic, nr_m_topic)

(9, 94, 0)

In [69]:
# CHECK LAZY LEARNER
#for i in range(nr_w_topic):
#    print(re.findall(topic_regex, texts[w_topic[i]])) #Sanity check

In [78]:
re.findall(r'[\\]n[\\]n.+?[\\]{1,2}n\d\s-\s.+?[\\]{1,2}n[\\]n', texts[w_topic[1]])

['\\n\\nConsejo de Ancianos Rapa Nui (organización tradicional no registrada) \\nParlamento Rapa Nui (organización tradicional no registrada) \\nHonui (organización tradicional no registrada)\\n\\nAutonomías territoriales e indígenas\\n3 - Forma de Estado, Ordenamiento, Autonomía, Descentralización, Equidad, Justicia Territorial, Gobiernos Locales y Organización Fiscal\\n\\n']

In [79]:
ind_df = pd.read_csv('Data/ind_docs.csv', names=['text'], skiprows=[0])
pop_df = pd.read_csv('Data/pop_docs.csv', names=['text'], skiprows=[0])
conv_df = pd.read_csv('Data/conv_docs.csv', names=['text'], skiprows=[0])
mult_ind_df = pd.read_csv('Data/mult_ind_docs.csv', names=['text'], skiprows=[0])
mult_pop_df = pd.read_csv('Data/pop_mult_docs.csv', names=['text'], skiprows=[0])

In [80]:
dfs = [mult_pop_df, mult_ind_df, conv_df, pop_df, ind_df]

In [82]:
mult_pop_df.shape

(25, 1)

In [83]:
nr_docs = 0
for df in dfs:
    nr_docs += df.shape[0]

In [152]:
corpus = []
for df in dfs:
    [corpus.append(spa_char(doc).split('\\n\\n')) for doc in df.text]

In [153]:
corpus_no_s = []
for doc in corpus:
    corpus_no_s.append([re.sub(r'\\n', r' ', paragraph) for paragraph in doc])

In [142]:
len(corpus[0])

22

In [143]:
len(doc_no_s[0])

22

In [154]:
corpus_no_s[:5]

[['',
  'La carencia de principios estructuradores que definan la movilidad ha permitido la improvisación, la prevalencia de intereses, las interpretaciones no fundamentadas o las respuestas disímiles frente a un mismo problema en la planificación de la movilidad del país.  En los últimos decenios, la movilidad se ha construido desde el vehículo motorizado particular, con esporádicas iniciativas en pro del transporte público e intermodal principalmente en las ciudades capitales y la presencia de espacios segregados para peatones y ciclistas. Sin embargo, estos no cumplen con estándares adecuados, debido a que estos ni siquiera existen, además de no abordar todo el territorio. La presencia de intereses económicos para la eliminación o reducción de una calzada, área verde o ciclovía; los disímiles criterios que aplica una municipalidad sobre lo que es una vía para bicicletas; el lobby que realizan empresas frente a una norma ministerial o al trámite de proyectos que privilegien al peatón

In [127]:
re.findall(r'\\n', corpus[1][0])

['\\n', '\\n', '\\n', '\\n', '\\n']

In [130]:
corpus[1][0]

'Existe un creciente cuestionamiento a la eficiencia y eficacia del Poder Legislativo en Chile, donde se\\nresponsabiliza erróneamente a la actual estructura bicameral del Congreso Nacional. La falta de eficacia y\\neficiencia del Poder Legislativo en Chile no radica en su estructura bicameral, sino más bien en la cantidad\\nde legisladores y su representatividad, junto con su propio funcionamiento orgánico, organización interna y\\nel desempeño mismo de los legisladores. Se ha propuesto un régimen unicameral obviando sus enormes\\ndificultades y desventajas. '

In [106]:
spa_char(mult_pop_df.iloc[0,0]).split('\\n\\n')

['',
 'La carencia de principios estructuradores que definan la movilidad ha permitido la improvisación, la\\nprevalencia de intereses, las interpretaciones no fundamentadas o las respuestas disímiles frente a un\\nmismo problema en la planificación de la movilidad del país. \\nEn los últimos decenios, la movilidad se ha construido desde el vehículo motorizado particular, con\\nesporádicas iniciativas en pro del transporte público e intermodal principalmente en las ciudades capitales\\ny la presencia de espacios segregados para peatones y ciclistas. Sin embargo, estos no cumplen con\\nestándares adecuados, debido a que estos ni siquiera existen, además de no abordar todo el territorio.\\nLa presencia de intereses económicos para la eliminación o reducción de una calzada, área verde o\\nciclovía; los disímiles criterios que aplica una municipalidad sobre lo que es una vía para bicicletas; el\\nlobby que realizan empresas frente a una norma ministerial o al trámite de proyectos que privi

In [91]:
corpus[:3]

[['',
  '',
  'La carencia de principios estructuradores que de\\xef\\xac\\x81nan la movilidad ha permitido la improvisación, la',
  'prevalencia de intereses, las interpretaciones no fundamentadas o las respuestas disímiles frente a un',
  'mismo problema en la plani\\xef\\xac\\x81cación de la movilidad del país. ',
  'En los últimos decenios, la movilidad se ha construido desde el vehículo motorizado particular, con',
  'esporádicas iniciativas en pro del transporte público e intermodal principalmente en las ciudades capitales',
  'y la presencia de espacios segregados para peatones y ciclistas. Sin embargo, estos no cumplen con',
  'estándares adecuados, debido a que estos ni siquiera existen, además de no abordar todo el territorio.',
  'La presencia de intereses económicos para la eliminación o reducción de una calzada, área verde o',
  'ciclovía; los disímiles criterios que aplica una municipalidad sobre lo que es una vía para bicicletas; el',
  'lobby que realizan empresas frent